# 16장

In [1]:
import tensorflow as tf, os
import warnings
warnings.filterwarnings('ignore')

tf.get_logger().setLevel('ERROR')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 방지 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
os.putenv('TF_GPU_ALLOCATOR', 'cuda_malloc_async')

In [2]:
import numpy as np, tensorflow as tf
from tensorflow import keras

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[10 11 12 13]
 [ 8  9 10 11]
 [ 0  1  2  3]]
===== 
Y_batch
[[11 12 13 14]
 [ 9 10 11 12]
 [ 1  2  3  4]]
____________________ Batch 1 
X_batch
[[2 3 4 5]
 [6 7 8 9]
 [4 5 6 7]]
===== 
Y_batch
[[ 3  4  5  6]
 [ 7  8  9 10]
 [ 5  6  7  8]]


In [3]:
for i in tf.data.Dataset.from_tensor_slices(tf.range(15)):
    print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)


In [4]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
print(tokenizer.texts_to_sequences(['First']))
print(tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(['First'])))
max_id = len(tokenizer.word_index)  # 고유한 문자 갯수
dataset_size = tokenizer.document_count # 전체 문자 갯수
print(max_id, dataset_size)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) -1   # 1~39를 0~38로 변환하기 위해 -1
print(encoded.shape)

[[20, 6, 9, 8, 3]]
['f i r s t']
39 1115394
(1115394,)


In [6]:
# train_size = dataset_size * 90 // 100
train_size = dataset_size * 90 // 50000
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
print(train_size, dataset_size)

2007 1115394


In [7]:
n_steps = 100
window_length = n_steps + 1 # target: 다음 1글자 앞의 input
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [9]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
60/60 [==============================] - 41s 619ms/step - loss: 3.1098
Epoch 2/10
60/60 [==============================] - 39s 642ms/step - loss: 2.5213
Epoch 3/10
60/60 [==============================] - 46s 769ms/step - loss: 2.1505
Epoch 4/10
60/60 [==============================] - 44s 735ms/step - loss: 1.9232
Epoch 5/10
60/60 [==============================] - 41s 681ms/step - loss: 1.6984
Epoch 6/10
60/60 [==============================] - 47s 788ms/step - loss: 1.4702
Epoch 7/10
60/60 [==============================] - 43s 714ms/step - loss: 1.2467
Epoch 8/10
60/60 [==============================] - 46s 760ms/step - loss: 1.0494
Epoch 9/10
60/60 [==============================] - 47s 785ms/step - loss: 0.8860
Epoch 10/10
60/60 [==============================] - 56s 931ms/step - loss: 0.7547


In [11]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are yo"])
y_pred = np.argmax(model(X_new), axis=-1)
print(X_new)
print()
print(y_pred)
print(tokenizer.sequences_to_texts(y_pred + 1)[0][-1])

tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [12]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1

    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    
    return text

In [13]:
print(complete_text('t', temperature=0.2))
print('='*30)
print(complete_text('w', temperature=1))
print('='*30)
print(complete_text('w', temperature=2))

t the object of our misery, is as an
inventory to p
w bat sole, to the commontly.

second citizen:
wear
w ze humsnll to lar ness't ai
, but they, bf tye is


In [14]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [15]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2, 
                     batch_input_shape=[1, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
20/20 [==============================] - 9s 324ms/step - loss: 3.4085
Epoch 2/50
20/20 [==============================] - 6s 321ms/step - loss: 3.0826
Epoch 3/50
20/20 [==============================] - 6s 320ms/step - loss: 3.0255
Epoch 4/50
20/20 [==============================] - 7s 345ms/step - loss: 2.9721
Epoch 5/50
20/20 [==============================] - 7s 331ms/step - loss: 2.8912
Epoch 6/50
20/20 [==============================] - 7s 337ms/step - loss: 2.7755
Epoch 7/50
20/20 [==============================] - 7s 337ms/step - loss: 2.6375
Epoch 8/50
20/20 [==============================] - 7s 343ms/step - loss: 2.5332
Epoch 9/50
20/20 [==============================] - 7s 376ms/step - loss: 2.4522
Epoch 10/50
20/20 [==============================] - 8s 408ms/step - loss: 2.4032
Epoch 11/50
20/20 [==============================] - 7s 359ms/step - loss: 2.3530
Epoch 12/50
20/20 [==============================] - 7s 374ms/step - loss: 2.2816
Epoch 13/50
20/20 [======

In [16]:
print(complete_text('t', temperature=0.2))
print('='*30)
print(complete_text('w', temperature=1))
print('='*30)
print(complete_text('w', temperature=2))

tizen:
what he paties in what he cane to that he ha
wag is fur aciun:
whll is revent: th prouslf
is cou
wen:
wh msmere
?
cul
lusu
ianthnn.
woud,spit$ug,?
w


In [10]:
import tensorflow as tf, os, numpy as np
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

tf.get_logger().setLevel('ERROR')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 방지 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
os.putenv('TF_GPU_ALLOCATOR', 'cuda_malloc_async')

In [11]:
import tensorflow_datasets as tfds

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token

print(" ".join(id_to_word[id_] for id_ in X_train[0][:10]))

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

(25000,) (25000,) (25000,) (25000,)
<sos> this film was just brilliant casting location scenery story


In [12]:
from collections import Counter

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b'<br\\s*/?>', b' ')
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b' ')
    X_batch = tf.strings.split(X_batch)

    return X_batch.to_tensor(default_value=b'<pad>'), y_batch

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

print(vocabulary.most_common()[:3])

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]


In [13]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [14]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [15]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # 0 값을 패딩으로 간주하여 무시
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 41s 47ms/step - loss: 0.5397 - accuracy: 0.7184
Epoch 2/5
782/782 [==============================] - 43s 54ms/step - loss: 0.3398 - accuracy: 0.8593
Epoch 3/5
782/782 [==============================] - 43s 55ms/step - loss: 0.1874 - accuracy: 0.9330
Epoch 4/5
782/782 [==============================] - 38s 49ms/step - loss: 0.1420 - accuracy: 0.9484
Epoch 5/5
782/782 [==============================] - 41s 53ms/step - loss: 0.0996 - accuracy: 0.9637


In [16]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
# 0=False 나머지는 True로 하여 패딩값 무시하도록 함.
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 47s 55ms/step - loss: 0.5439 - accuracy: 0.7202
Epoch 2/5
782/782 [==============================] - 43s 54ms/step - loss: 0.3751 - accuracy: 0.8380
Epoch 3/5
782/782 [==============================] - 44s 57ms/step - loss: 0.2256 - accuracy: 0.9162
Epoch 4/5
782/782 [==============================] - 43s 55ms/step - loss: 0.1390 - accuracy: 0.9502
Epoch 5/5
782/782 [==============================] - 43s 55ms/step - loss: 0.1331 - accuracy: 0.9481


In [17]:
import tensorflow_hub as hub

model = keras.models.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/nnlm-en-dim50/1',
                   dtype=tf.string, input_shape=[], output_shape=50),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
batch_size = 32
train_set = datasets['train'].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5459 - accuracy: 0.7263
Epoch 2/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5147 - accuracy: 0.7478
Epoch 3/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5093 - accuracy: 0.7505
Epoch 4/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5057 - accuracy: 0.7537
Epoch 5/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5027 - accuracy: 0.7551


In [18]:
model = keras.Sequential([
    # trainable=True로 할 경우 코랩에서 메모리 부족 에러가 발생합니다.
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=False, dtype=tf.string, input_shape=[]),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=10);

Epoch 1/10
782/782 [==============================] - 19s 23ms/step - loss: 0.3756 - accuracy: 0.8406
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3251 - accuracy: 0.8593
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3201 - accuracy: 0.8627
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3157 - accuracy: 0.8656
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3113 - accuracy: 0.8675
Epoch 6/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3066 - accuracy: 0.8695
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3019 - accuracy: 0.8722
Epoch 8/10
782/782 [==============================] - 19s 24ms/step - loss: 0.2972 - accuracy: 0.8751
Epoch 9/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2922 - accuracy: 0.8766
Epoch 10/10
782/782 [==============================] - 18s 23ms/step - loss: 0.287

In [20]:
import tensorflow_addons as tfa
import numpy as np

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings, initial_state=encoder_state, 
                                                             sequence_length=sequence_lengths)
y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths], outputs=[y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

X = np.random.randint(100, size=10*1000).reshape(1000, 10)
y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], y, epochs=2)

Epoch 1/2
32/32 [==============================] - 7s 158ms/step - loss: 6.1568
Epoch 2/2
32/32 [==============================] - 5s 160ms/step - loss: 4.6670


In [ ]:
# beam_width = 10
# decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
#     cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)
# decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(encoder_state, multiplier=beam_width)
# outputs, _, _ = decoder(embedding_decoder, start_tokens=start_tokens, end_token=end_token, 
#                         initial_state=decoder_initial_state)

In [1]:
import tensorflow as tf, os, numpy as np
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

tf.get_logger().setLevel('ERROR')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 방지 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
os.putenv('TF_GPU_ALLOCATOR', 'cuda_malloc_async')

In [2]:
from pathlib import Path

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # 쌍을 2개의 리스트로 분리합니다.

for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

I have no further questions. => No tengo más preguntas.
Tom is being hunted by the police. => Tom está siendo perseguido por la policía.
I want to go to bed early. => Quiero irme temprano a la cama.


In [3]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

print(text_vec_layer_en.get_vocabulary()[:10])
print(text_vec_layer_es.get_vocabulary()[:10])

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']
['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']


In [4]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

encoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 116s 34ms/step - loss: 0.4154 - accuracy: 0.4248 - val_loss: 0.3110 - val_accuracy: 0.5223
Epoch 2/10
3125/3125 [==============================] - 133s 42ms/step - loss: 0.2654 - accuracy: 0.5718 - val_loss: 0.2394 - val_accuracy: 0.6044
Epoch 3/10
3125/3125 [==============================] - 144s 46ms/step - loss: 0.2087 - accuracy: 0.6430 - val_loss: 0.2090 - val_accuracy: 0.6448
Epoch 4/10
3125/3125 [==============================] - 147s 47ms/step - loss: 0.1760 - accuracy: 0.6873 - val_loss: 0.1947 - val_accuracy: 0.6659
Epoch 5/10
3125/3125 [==============================] - 147s 47ms/step - loss: 0.1528 - accuracy: 0.7203 - val_loss: 0.1878 - val_accuracy: 0.6746
Epoch 6/10
3125/3125 [==============================] - 130s 42ms/step - loss: 0.1342 - accuracy: 0.7479 - val_loss: 0.1859 - val_accuracy: 0.6809
Epoch 7/10
3125/3125 [==============================] - 97s 31ms/step - loss: 0.1181 - accuracy: 0.7731 - val_loss: 0.

In [5]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input 
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

print(translate("I like soccer"))
print('='*30)
print(translate("I like soccer and also going to the beach"))

1/1 [==============================] - 0s 20ms/step
me gusta el fútbol
1/1 [==============================] - 0s 25ms/step
me gusta [UNK] y a la playa no me gusta


In [6]:
encoder = keras.layers.Bidirectional(
    keras.layers.LSTM(256, return_state=True))
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # 단기 상태 (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # 장기 상태 (1 & 3)
# 추가 코드 - 모델을 완성하고 학습시킵니다.
decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))
print(translate("I like soccer"))
print('='*30)
print(translate("I like soccer and also going to the beach"))

Epoch 1/10
3125/3125 [==============================] - 122s 36ms/step - loss: 0.3101 - accuracy: 0.5366 - val_loss: 0.2219 - val_accuracy: 0.6288
Epoch 2/10
3125/3125 [==============================] - 109s 35ms/step - loss: 0.1923 - accuracy: 0.6682 - val_loss: 0.1884 - val_accuracy: 0.6751
Epoch 3/10
3125/3125 [==============================] - 109s 35ms/step - loss: 0.1604 - accuracy: 0.7119 - val_loss: 0.1766 - val_accuracy: 0.6910
Epoch 4/10
3125/3125 [==============================] - 109s 35ms/step - loss: 0.1393 - accuracy: 0.7414 - val_loss: 0.1732 - val_accuracy: 0.6952
Epoch 5/10
3125/3125 [==============================] - 113s 36ms/step - loss: 0.1226 - accuracy: 0.7664 - val_loss: 0.1717 - val_accuracy: 0.6998
Epoch 6/10
3125/3125 [==============================] - 118s 38ms/step - loss: 0.1082 - accuracy: 0.7894 - val_loss: 0.1754 - val_accuracy: 0.6992
Epoch 7/10
3125/3125 [==============================] - 111s 35ms/step - loss: 0.0960 - accuracy: 0.8092 - val_loss: 0

In [7]:
# 추가 코드 - 빔 검색의 기본 구현

def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # 인코더 입력
    X_dec = np.array(["startofseq"])  # 디코더 입력
    y_proba = model.predict((X, X_dec))[0, 0]  # 첫 번째 토큰의 확률
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # 촤상의 (log_proba, translation) 리스트 
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]
    
    # 추가 코드 - verbose 모드에서 상위 첫 단어를 표시합니다.
    if verbose:
        print("상위 첫 단어:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # 번역이 완료되었으므로 번역을 이어가지 않습니다.
            X = np.array([sentence_en])  # 인코더 입력
            X_dec = np.array(["startofseq " + translation])  # 디코더 입력
            y_proba = model.predict((X, X_dec))[0, idx]  # 마지막 토큰의 확률
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # 추가 코드 - verbose 모드의 경우 지금까지의 최상의 번역을 출력합니다.
        if verbose:
            print("지금까지 최상의 번역:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

# 추가 코드 - 모델이 어떻게 오류를 발생시키는지 보여줍니다.
sentence_en = "I love cats and dogs"
print(translate(sentence_en))
print('='*30)
print(beam_search(sentence_en, beam_width=3, verbose=True))

1/1 [==============================] - 0s 39ms/step
me gustan los perros y los gatos
1/1 [==============================] - 0s 38ms/step
상위 첫 단어: [(-0.03524482, 'me'), (-4.074361, 'yo'), (-4.891246, 'amo')]
1/1 [==============================] - 0s 48ms/step
지금까지 최상의 번역: [(-0.17349973, 'me gustan'), (-2.3978343, 'me [UNK]'), (-3.8907247, 'me encanta')]
1/1 [==============================] - 0s 61ms/step
지금까지 최상의 번역: [(-0.44391635, 'me gustan los'), (-1.7447308, 'me gustan tanto'), (-2.5370724, 'me [UNK] tanto')]
1/1 [==============================] - 0s 76ms/step
지금까지 최상의 번역: [(-1.0484011, 'me gustan los perros'), (-1.235138, 'me gustan los gatos'), (-1.857966, 'me gustan tanto los')]
1/1 [==============================] - 0s 48ms/step
지금까지 최상의 번역: [(-1.0535488, 'me gustan los perros y'), (-1.2409387, 'me gustan los gatos y'), (-1.8744655, 'me gustan tanto los perros')]
1/1 [==============================] - 0s 51ms/step
지금까지 최상의 번역: [(-1.093529, 'me gustan los perros y los'), (-1.2701

In [8]:
encoder = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True, return_state=True))

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),
                 tf.concat(encoder_state[1::2], axis=-1)]
decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

attention_layer = keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = keras.layers.Dense(vocab_size, activation='softmax')
y_proba = output_layer(attention_outputs)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 134s 40ms/step - loss: 0.3024 - accuracy: 0.5544 - val_loss: 0.2100 - val_accuracy: 0.6566
Epoch 2/10
3125/3125 [==============================] - 125s 40ms/step - loss: 0.1888 - accuracy: 0.6819 - val_loss: 0.1866 - val_accuracy: 0.6881
Epoch 3/10
3125/3125 [==============================] - 116s 37ms/step - loss: 0.1656 - accuracy: 0.7130 - val_loss: 0.1774 - val_accuracy: 0.6995
Epoch 4/10
3125/3125 [==============================] - 121s 39ms/step - loss: 0.1497 - accuracy: 0.7345 - val_loss: 0.1738 - val_accuracy: 0.7053
Epoch 5/10
3125/3125 [==============================] - 122s 39ms/step - loss: 0.1370 - accuracy: 0.7530 - val_loss: 0.1739 - val_accuracy: 0.7079
Epoch 6/10
3125/3125 [==============================] - 118s 38ms/step - loss: 0.1260 - accuracy: 0.7689 - val_loss: 0.1747 - val_accuracy: 0.7113
Epoch 7/10
3125/3125 [==============================] - 120s 38ms/step - loss: 0.1167 - accuracy: 0.7828 - val_loss: 0

In [9]:
print(translate("I like soccer and also going to the beach"))
print('='*30)
print(beam_search("I like soccer and also going to the beach", beam_width=3, verbose=True))

1/1 [==============================] - 0s 27ms/step
me gusta el fútbol y también ir a la playa
1/1 [==============================] - 0s 24ms/step
상위 첫 단어: [(-0.074255265, 'me'), (-4.6216154, 'yo'), (-4.801601, 'el')]
1/1 [==============================] - 0s 51ms/step
지금까지 최상의 번역: [(-0.090356335, 'me gusta'), (-4.5053167, 'me encanta'), (-4.6623745, 'yo gusta')]
1/1 [==============================] - 0s 55ms/step
지금까지 최상의 번역: [(-0.2396906, 'me gusta el'), (-3.2079437, 'me gusta fútbol'), (-3.333143, 'me gusta [UNK]')]
1/1 [==============================] - 0s 50ms/step
지금까지 최상의 번역: [(-0.24173188, 'me gusta el fútbol'), (-3.2113903, 'me gusta fútbol y'), (-3.8187408, 'me gusta [UNK] y')]
1/1 [==============================] - 0s 56ms/step
지금까지 최상의 번역: [(-0.24807271, 'me gusta el fútbol y'), (-3.2279015, 'me gusta fútbol y también'), (-3.9083428, 'me gusta [UNK] y también')]
1/1 [==============================] - 0s 54ms/step
지금까지 최상의 번역: [(-0.29714185, 'me gusta el fútbol y también'), 

In [10]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims%2 == 1:
            max_dims += 1   # max_dims는 짝수여야 함
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims//2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    
    def call(self, inputs):
        shape = tf.shape(inputs)
        
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [11]:
embed_size = 512
max_steps = 500
vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Z = encoder_in
for N in range(6):
    X = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    # Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, Z], use_causal_mask=True)    # tensorflow WARNING에 따라 변경
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(keras.layers.Dense(vocab_size, activation='softmax'))(Z)

In [12]:
max_length = 50  # 전체 훈련 세트에 있는 최대 길이
embed_size = 128

encoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

pos_embed_layer = keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

In [13]:
N = 2
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    
    skip = Z
    Z = keras.layers.Dense(n_units, activation='relu')(Z)
    Z = keras.layers.Dense(embed_size)(Z)
    Z = keras.layers.Dropout(dropout_rate)(Z)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))

In [14]:
batch_max_len_dec = 50  # tf.shape(decoder_embeddings)[1]
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, None]
causal_mask = tf.linalg.band_part(  # 행렬의 대각선 부분을 추출하거나 수정하는 함수
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

encoder_outputs = Z
Z = decoder_in
for _ in range(N):
    skip = Z
    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask&decoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    skip = Z

    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    skip = Z

    Z = keras.layers.Dense(n_units, activation='relu')(Z)
    Z = keras.layers.Dense(embed_size)(Z)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))

y_proba = keras.layers.Dense(vocab_size, activation='softmax')(Z)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 406s 128ms/step - loss: 1.0423 - accuracy: 0.8620 - val_loss: 0.6979 - val_accuracy: 0.8774
Epoch 2/10
3125/3125 [==============================] - 409s 131ms/step - loss: 0.7588 - accuracy: 0.8832 - val_loss: 0.5549 - val_accuracy: 0.9011
Epoch 3/10
3125/3125 [==============================] - 404s 129ms/step - loss: 0.7956 - accuracy: 0.8807 - val_loss: 1.0742 - val_accuracy: 0.8583
Epoch 4/10
3125/3125 [==============================] - 394s 126ms/step - loss: 0.7040 - accuracy: 0.8840 - val_loss: 0.6085 - val_accuracy: 0.8946
Epoch 5/10
3125/3125 [==============================] - 397s 127ms/step - loss: 0.5611 - accuracy: 0.8989 - val_loss: 0.5584 - val_accuracy: 0.9006
Epoch 6/10
3125/3125 [==============================] - 395s 127ms/step - loss: 0.5029 - accuracy: 0.9050 - val_loss: 0.4864 - val_accuracy: 0.9066
Epoch 7/10
3125/3125 [==============================] - 396s 127ms/step - loss: 0.4726 - accuracy: 0.9075 - val_

In [15]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # 다른 많은 작업을 사용할 수 있습니다.
result = classifier("The actors were very convincing.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
classifier(["I am from India.", "I am from Iraq."])

[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

In [17]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'contradiction', 'score': 0.9790191650390625}]

In [18]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [19]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!",
                       "Joe lived for a very long time. [SEP] Joe is old."],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]])>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [20]:
token_ids = tokenizer([("I like soccer.", "We all love soccer!"),
                       ("Joe lived for a very long time.", "Joe is old.")],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]])>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [21]:
outputs = model(token_ids)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-2.110418  ,  1.1792463 ,  1.4074315 ],
       [-0.01550994,  1.0979388 , -0.99263567]], dtype=float32)>, hidden_states=None, attentions=None)

In [22]:
Y_probas = keras.activations.softmax(outputs.logits)
Y_probas

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.01624809, 0.4359988 , 0.54775316],
       [0.22618128, 0.6886861 , 0.08513268]], dtype=float32)>

In [23]:
Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred  # 0 = contradiction, 1 = entailment, 2 = neutral

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1], dtype=int64)>

In [ ]:
sentences = [("Sky is blue", "Sky is red"), ("I love her", "She loves me")]
X_train = tokenizer(sentences, padding=True, return_tensors="tf").data
y_train = tf.constant([0, 2])  # contradiction, neutral
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2)

In [29]:
import numpy as np, keras, tensorflow as tf

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

MONTHS_DICT = {f'{i:>02}': v for i, v in enumerate(MONTHS, 1)}
print(MONTHS_DICT)

start_date = np.datetime64('1000-01-01')
end_date = np.datetime64('9999-12-31')
y = np.arange(start_date, end_date + np.timedelta64(1, 'D'), dtype='datetime64[D]')
np.random.shuffle(y)
y = list(map(str, y))
X = list(map(lambda x: MONTHS_DICT[x[5:7]] + ' ' + x[-2:] + ', ' + x[:4], y))
print(X[0], len(X), y[0], len(y))

{'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'}
November 11, 4281 3287182 4281-11-11 3287182


In [30]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
# MONTHS를 join 후 set으로 중복 문자 제거한 후 숫자와 쉼표, 공백을 포함하여 입력값 원소를 정렬하여 문자열로 생성
print(INPUT_CHARS)
OUTPUT_CHARS = "0123456789-"    # 출력값 원소를 문자열로 생성

 ,0123456789ADFJMNOSabceghilmnoprstuvy


In [31]:
date_str_to_ids = lambda x:list(map(OUTPUT_CHARS.index, x))
def prepare_date_strs(date_strs, date_str_to_ids=date_str_to_ids):
    X_ids = list(map(date_str_to_ids, date_strs))
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # 0을 패딩 토큰 ID로 사용

Y_train = prepare_date_strs(y[:10000])
Y_valid = prepare_date_strs(y[10000:12000])
Y_test = prepare_date_strs(y[12000:14000])

date_str_to_ids = lambda x:list(map(INPUT_CHARS.index, x))
def prepare_date_strs(date_strs, date_str_to_ids=date_str_to_ids):
    X_ids = list(map(date_str_to_ids, date_strs))
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # 0을 패딩 토큰 ID로 사용

X_train = prepare_date_strs(X[:10000])
X_valid = prepare_date_strs(X[10000:12000])
X_test = prepare_date_strs(X[12000:14000])

In [32]:
embedding_size = 32
max_output_length = 10

model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS)+1, output_dim=embedding_size, input_shape=[None]),
    keras.layers.LSTM(128),
    keras.layers.RepeatVector(max_output_length),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS)+1, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 32)          1248      
                                                                 
 lstm_6 (LSTM)               (None, 128)               82432     
                                                                 
 repeat_vector (RepeatVector  (None, 10, 128)          0         
 )                                                               
                                                                 
 lstm_7 (LSTM)               (None, 10, 128)           131584    
                                                                 
 dense_13 (Dense)            (None, 10, 12)            1548      
                                                                 
Total params: 216,812
Trainable params: 216,812
Non-trainable params: 0
__________________________________________________

In [33]:
model.compile('nadam', 'sparse_categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, Y_train, 32, 20, validation_data=(X_valid, Y_valid))
model.evaluate(X_test, Y_test)

Epoch 1/20
313/313 [==============================] - 6s 15ms/step - loss: 1.8064 - accuracy: 0.3476 - val_loss: 1.4035 - val_accuracy: 0.4661
Epoch 2/20
313/313 [==============================] - 4s 12ms/step - loss: 1.3494 - accuracy: 0.5127 - val_loss: 1.1450 - val_accuracy: 0.5968
Epoch 3/20
313/313 [==============================] - 4s 12ms/step - loss: 1.1790 - accuracy: 0.5905 - val_loss: 0.9765 - val_accuracy: 0.6460
Epoch 4/20
313/313 [==============================] - 4s 11ms/step - loss: 0.8473 - accuracy: 0.6849 - val_loss: 0.7362 - val_accuracy: 0.7140
Epoch 5/20
313/313 [==============================] - 3s 11ms/step - loss: 0.6501 - accuracy: 0.7463 - val_loss: 0.5763 - val_accuracy: 0.7707
Epoch 6/20
313/313 [==============================] - 4s 11ms/step - loss: 0.6280 - accuracy: 0.7674 - val_loss: 0.4504 - val_accuracy: 0.8293
Epoch 7/20
313/313 [==============================] - 4s 12ms/step - loss: 0.4239 - accuracy: 0.8437 - val_loss: 0.3076 - val_accuracy: 0.8802

[0.0029835947789251804, 1.0]

In [34]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])
ids = model.predict(X_new).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)


def prepare_date_strs_padded(date_strs, max_length=X_train.shape[1]):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_length:
        X = tf.pad(X, [[0, 0], [0, max_length - X.shape[1]]])
    
    return X

X_new = prepare_date_strs_padded(["May 02, 2020", "July 14, 1789"])
ids = model.predict(X_new).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1/1 [==============================] - 0s 430ms/step
2020-02-02
1789-02-14
1/1 [==============================] - 0s 449ms/step
2020-05-02
1789-07-14


In [35]:
import numpy as np, keras, tensorflow as tf

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

MONTHS_DICT = {f'{i:>02}': v for i, v in enumerate(MONTHS, 1)}
print(MONTHS_DICT)

start_date = np.datetime64('1000-01-01')
end_date = np.datetime64('9999-12-31')
y = np.arange(start_date, end_date + np.timedelta64(1, 'D'), dtype='datetime64[D]')
np.random.shuffle(y)
y = list(map(str, y))
X = list(map(lambda x: MONTHS_DICT[x[5:7]] + ' ' + x[-2:] + ', ' + x[:4], y))


print(X[0], len(X), y[0], len(y))
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
# MONTHS를 join 후 set으로 중복 문자 제거한 후 숫자와 쉼표, 공백을 포함하여 입력값 원소를 정렬하여 문자열로 생성
print(INPUT_CHARS)
OUTPUT_CHARS = "0123456789-"    # 출력값 원소를 문자열로 생성


date_str_to_ids = lambda x:list(map(OUTPUT_CHARS.index, x))
def prepare_date_strs(date_strs, date_str_to_ids=date_str_to_ids):
    X_ids = list(map(date_str_to_ids, date_strs))
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # 0을 패딩 토큰 ID로 사용

Y_train = prepare_date_strs(y[:10000])
Y_valid = prepare_date_strs(y[10000:12000])
Y_test = prepare_date_strs(y[12000:14000])

date_str_to_ids = lambda x:list(map(INPUT_CHARS.index, x))
def prepare_date_strs(date_strs, date_str_to_ids=date_str_to_ids):
    X_ids = list(map(date_str_to_ids, date_strs))
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # 0을 패딩 토큰 ID로 사용

X_train = prepare_date_strs(X[:10000])
X_valid = prepare_date_strs(X[10000:12000])
X_test = prepare_date_strs(X[12000:14000])

{'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'}
August 25, 1918 3287182 1918-08-25 3287182
 ,0123456789ADFJMNOSabceghilmnoprstuvy


In [36]:
sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [37]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1,output_dim=encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2, output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state)

decoder_output = keras.layers.Dense(len(OUTPUT_CHARS) + 1,
                                    activation="softmax")(decoder_lstm_output)

model = keras.Model(inputs=[encoder_input, decoder_input],
                           outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 32)     1248        ['input_7[0][0]']                
                                                                                                  
 embedding_8 (Embedding)        (None, None, 32)     416         ['input_8[0][0]']                
                                                                                            

In [38]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 7s 16ms/step - loss: 1.6888 - accuracy: 0.3598 - val_loss: 1.4800 - val_accuracy: 0.4083
Epoch 2/10
313/313 [==============================] - 4s 13ms/step - loss: 1.3470 - accuracy: 0.4816 - val_loss: 1.1992 - val_accuracy: 0.5470
Epoch 3/10
313/313 [==============================] - 4s 13ms/step - loss: 0.9443 - accuracy: 0.6512 - val_loss: 0.6522 - val_accuracy: 0.7677
Epoch 4/10
313/313 [==============================] - 4s 14ms/step - loss: 0.4352 - accuracy: 0.8586 - val_loss: 0.2382 - val_accuracy: 0.9395
Epoch 5/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1397 - accuracy: 0.9762 - val_loss: 0.0755 - val_accuracy: 0.9932
Epoch 6/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0496 - accuracy: 0.9976 - val_loss: 0.0383 - val_accuracy: 0.9977
Epoch 7/10
313/313 [==============================] - 5s 14ms/step - loss: 0.0734 - accuracy: 0.9888 - val_loss: 0.0256 - val_accuracy: 0.9994

In [39]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
    print(Y_pred)
    return ids_to_date_strs(Y_pred[:, 1:])

predict_date_strs(["July 14, 1789", "May 01, 2020"])

1/1 [==============================] - 0s 25ms/step
tf.Tensor(
[[12  2  8  9 10 11  1  8 11  2  5]
 [12  3  1  3  1 11  1  6 11  1  2]], shape=(2, 11), dtype=int32)


['1789-07-14', '2020-05-01']